In [25]:
import numpy as np
import pandas as pd
import seaborn as sns
import subprocess
import matplotlib.pyplot as plt
import sys
import os
from fnmatch import fnmatch

In [26]:
def listfiles(root, pattern):
  for path, subdirs, files in os.walk(root):
    for name in files:
      if fnmatch(name, pattern):
        yield os.path.join(path, name)

# EXPERIMENTOS CUANTITATIVOS

## Diferencia de resultados

Tenemos 3 metodos:
   - CMM
   - WP
   - WP con Laplace

Para cada método, ejecutamos un conjunto de test supervisados y comparamos el modulo del error obtenido en cada uno.
Para CMM usamos los tests de la catedra. En los otros dos, los generamos utilizando los inputs de la catedra, pero outputs calculados a mano.

In [27]:
def correr_metodo(inputPath, outputPath, metodo):
    result = subprocess.run([f'../cmake-build-debug/tp1', inputPath, outputPath, metodo],
                            stdout=subprocess.PIPE, encoding='ascii')
    return result

### CMM

In [28]:
#Runnea todos los test de la catedra y guarda las salidas
outFolder = './cuantitativo_diff/'
def runCMMDifference():
    for fname in listfiles('../tests', '*.in'):
        name = fname.split('/')[-1]
        #print(name[-1])
        correr_metodo(fname, outFolder + name.replace(".in", ".out"), '0')

In [21]:
#runCMMDifference();

# Experimentos

In [29]:
metodos = ["CMM", "WP", "WPL"]

## EXPERIMENTOS CUALITATIVOS

En este experimento vamos a ejecutar un set de datos de la temporada 2016 de la nba, sin la ultima fecha. Despues de eso, vamos a ver que ranking (de cada metodo) se ajusta mejor a los resultados reales en la ultima fecha.

In [30]:
def armarTabla(scores_path, next_scores_path):
    #Busco el resultado
    scores = pd.read_csv(scores_path, names=['Score'])
    #Busco los nombres de los equipos
    equipos = pd.read_csv('../data/nba_2016_teams.csv', names=['ID', 'Team'])
    #Join de los df
    rankings = equipos.join(scores).sort_values(by='Score', ascending=False)

    next_scores = pd.read_csv(next_scores_path)
    next_scores['Acerto'] = 'No'

    for partido in next_scores.itertuples():
        team1 = rankings[rankings['ID'] == partido.equipo1]
        team2 = rankings[rankings['ID'] == partido.equipo2]
        score1 = team1['Score']
        score2 = team2['Score']

        #Guardo el nombre de los equipos
        next_scores.at[partido.Index, 'Equipo 1'] = team1['Team'].values[0]
        next_scores.at[partido.Index, 'Equipo 2'] = team2['Team'].values[0]
        #Guardo el score que saco en el ranking cada equipo
        next_scores.at[partido.Index, 'Score equipo 1'] = team1['Score'].values[0]
        next_scores.at[partido.Index, 'Score equipo 2'] = team2['Score'].values[0]
        #Guardo el resultado del partido en 1 sola columna
        next_scores.at[partido.Index, 'Resultado'] = "%s - %s" %(partido.puntaje1, partido.puntaje2)

        if (partido.puntaje1 > partido.puntaje2 and float(score1) > float(score2)) or \
           (partido.puntaje1 < partido.puntaje2 and float(score1) < float(score2)):
            next_scores.at[partido.Index, 'Acerto'] = 'Si'

    next_scores = next_scores[['Equipo 1', 'Resultado', 'Equipo 2', 'Score equipo 1', 'Score equipo 2', 'Acerto']]
    return next_scores

### Experimento sin ultima fecha

In [31]:
export_path_posfix = '_cualitativo_sin_ultima_fecha.csv'

for i in range(len(metodos)):
    correr_metodo('../data/nba_2016_scores_without_lastdate.dat', metodos[i] + export_path_posfix, '%d' % i)
    
next_scores_path = '../data/nba_2016_scores_lastdate.csv'
for metodo in metodos:
    table = armarTabla(metodo + export_path_posfix, next_scores_path)
    table.to_csv('last_date_cualitativo_%s.csv' % metodo, index=False)

### Experimento primera segunda fecha

In [32]:
export_path_posfix = '_cualitativo_segunda_fecha.csv'

for i in range(len(metodos)):
    correr_metodo('../data/nba_2016_scores_first_date.dat', metodos[i] + export_path_posfix, '%d' % i)
    
next_scores_path = '../data/nba_2016_second_date.csv'
for metodo in metodos:
    table = armarTabla(metodo + export_path_posfix, next_scores_path)
    table.to_csv('segunda_fecha_cualitativo_%s.csv' % metodo, index=False)

### Experimentos de comparacion de metodos

In [41]:
export_path_posfix = '_cualitativo_segunda_fecha.csv'
CMM_scores = pd.read_csv(metodos[0] + export_path_posfix, names=['CMM score'])
WP_scores = pd.read_csv(metodos[1] + export_path_posfix, names=['WP score'])
WPL_scores = pd.read_csv(metodos[2] + export_path_posfix, names=['WPL score'])

equipos = pd.read_csv('../data/nba_2016_teams.csv', names=['ID', 'Team'])
#Join de los df
rankings = equipos.join(CMM_scores)
rankings = rankings.join(WP_scores)
rankings = rankings.join(WPL_scores)

rankings.to_csv('comparacion_metodos_cualitativo_segunda_fecha.csv', index=False)